<a href="https://colab.research.google.com/github/vishalsubbiah/agent_investor/blob/main/Copy_of_stitch_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass

KAY_API_KEY = getpass()
OPENAI_API_KEY = getpass()
ALPHAV_KEY = getpass()
ANTHROPIC_API_KEY = getpass()

··········
··········
··········
··········


In [ ]:
def generate_ticker_ideas(industry):
    system_prompt = f"You are a financial analyst assistant. Generate a list of 5 ticker symbols for major companies in the {industry} industry, as a Python-parseable list."

    messages = [
        {"role": "user", "content": f"Please provide a list of 5 ticker symbols for major companies in the {industry} industry as a Python-parseable list. Only respond with the list, no other text."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 200,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    ticker_list = ast.literal_eval(response_text)
    return [ticker.strip() for ticker in ticker_list]

def get_current_price(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period='1d', interval='1m')
    return data['Close'][-1]



In [ ]:
def get_search_results(search_term):
    url = f"https://serpapi.com/search.json?q={search_term}&api_key={SERP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data['organic_results']

In [ ]:
!pip install kay langchain_community langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import os

os.environ["KAY_API_KEY"] = KAY_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ALPHAV_KEY"] = ALPHAV_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [ ]:
class CompanyEarnings:

    def __init__(self, company_symbol):
        self._company_symbol = company_symbol
        #self._key_file = os.path.abspath(f"{__file__}/../../../.key")
        self._key = ALPHAV_KEY
        #with open(self._key_file) as f:
        #    self._key = f.read()

        self._url = "https://www.alphavantage.co/query"
        self._params = {
            "function": "EARNINGS",
            "symbol": self._company_symbol,
            "apikey": self._key,
            "datatype": "json"
        }
        self._response = None

    def _get_earnings(self):
        if self._response is None:
            self._response = requests.get(url=self._url, params=self._params).json()
            if dict(self._response).get("Note"):
                print(self._response)
                self._response = None
                return StatusResults.MinLimit
            elif dict(self._response).get("symbol") == self._company_symbol.upper() or len(dict(self._response)) == 0:
                return StatusResults.FreshlyCollected
        else:
            print(f"collected earnings already: {self._company_symbol}: {self._response}")
            return StatusResults.AlreadyCollected

    def save_results(self):
        file_path = os.path.abspath(f"{__file__}/../../../earnings_data/{self._company_symbol}.json")
        if not os.path.exists(file_path):
            sleep_counter = 0
            status = StatusResults.Default
            while status not in [StatusResults.FreshlyCollected, StatusResults.AlreadyCollected] and sleep_counter<2:
                status = self._get_earnings()
                if status == StatusResults.MinLimit:
                    print("triggering sleep")
                    time.sleep(60)
                    sleep_counter+=1
            # if sleep counter happens twice, it means we waited for atleast 1 min
            # but still failed to fetch due to api limit
            # so it must have hit day limit so inform that we need to wait for next day
            if sleep_counter<2:
                with open(file_path, "w") as outfile:
                    json.dump(self._response, outfile, indent=4)
                if len(dict(self._response)) == 0:
                    return StatusSaving.EmptyFile
                return StatusSaving.CreatedFile
            else:
                return StatusResults.DayLimit
        else:
            return StatusSaving.FileExists

In [ ]:
# Setup API keys for Kay and OpenAI
import os

from langchain_community.retrievers import KayAiRetriever

os.environ["KAY_API_KEY"] = KAY_API_KEY
retriever = KayAiRetriever.create(
    dataset_id="company", data_types=["10-K", "10-Q", "PressRelease"], num_contexts=3
)
docs = retriever.get_relevant_documents(
    "What were the biggest strategy changes and partnerships made by Roku in 2023??"
)

In [ ]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [ ]:
questions = [
    "What were the biggest strategy changes and partnerships made by Roku in 2023?"
    # "Where is Wex making the most money in 2023?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


-> **Question**: What were the biggest strategy changes and partnerships made by Roku in 2023? 

**Answer**: In 2023, Roku made significant strategy changes and partnerships, including:
1. Launching Roku-branded TVs to offer more choice and innovation to consumers and Roku TV partners.
2. Expanding its product line with smart home devices integrated into the Roku ecosystem.
3. Updating its operating system with more personalization features and enhancements.
4. Introducing new features like Photo Streams and Shoppable Ads to enhance the streaming experience.
5. Partnering with Warner Bros. Discovery to launch new channels and add on-demand content.
6. Expanding The Roku Channel with new content offerings, original movies, and exclusive content.
7. Partnering with Paramount+, Lionsgate, AMC Networks, and discovery+ to bring more content to viewers.

These are some of the notable strategy changes and partnerships made by Roku in 2023. 



In [ ]:
git push o